# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [6]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
output_data_file = "data_file/cities.csv"
city_data = pd.read_csv(output_data_file)

city_data_df = pd.DataFrame(city_data)
city_data_df.sample(20)


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
61,mataura,100,NZ,1609709968,80,-46.19,168.86,14.44,3.58
438,monrovia,20,LR,1609710344,94,6.30,-10.80,26.00,3.65
361,santa maria del oro,0,MX,1609710337,20,25.93,-105.37,19.62,2.67
45,sale,20,MA,1609710034,87,34.04,-6.82,10.00,3.50
508,karaman,39,TR,1609710348,82,37.08,33.25,2.46,0.44
26,rikitea,21,PF,1609709967,80,-23.12,-134.97,25.64,7.37
432,zaozerne,100,UA,1609710343,58,45.16,33.28,8.92,4.08
212,gamba,97,GA,1609709998,90,-2.65,10.00,25.19,2.25
562,tilichiki,100,RU,1609709979,92,60.47,166.10,-19.20,4.29
156,novikovo,96,RU,1609710321,97,51.15,37.89,-3.88,3.08


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
